In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.utils.data as Data
from torch.autograd import Variable
from imblearn.combine import SMOTEENN

In [263]:
#loading data
Non_MI_data = np.load('Non_MI_data.npy')
Non_MI_label = np.load('Non_MI_label.npy')
MI_generated_data = np.load('MI_generated_data.npy')
MI_generated_label = np.load('MI_generated_label.npy')
MI_data = np.load('MI_data.npy')
MI_label = np.load('MI_label.npy')

X_train_MI = np.load('MI_train_Kmeans.npy')
X_test_MI = np.load('MI_test_Kmeans.npy')
y_train_MI = np.load('MI_train_Kmeans_label.npy').tolist()
y_test_MI = np.load('MI_test_Kmeans_label.npy').tolist()
X_val_MI = np.load('MI_val_Kmeans.npy')
y_val_MI = np.load('MI_val_Kmeans_label.npy').tolist()


X_train_Non_MI = np.load('Non_MI_train_Kmeans.npy')
X_test_Non_MI = np.load('Non_MI_test_Kmeans.npy')
y_train_Non_MI = np.load('Non_MI_train_Kmeans_label.npy').tolist()
y_test_Non_MI = np.load('Non_MI_test_Kmeans_label.npy').tolist()
X_val_Non_MI = np.load('Non_MI_val_Kmeans.npy')
y_val_Non_MI = np.load('Non_MI_val_Kmeans_label.npy').tolist()


y_generatedMI = []
y_train_MI =[]

for i in MI_generated_data:
    y_generatedMI.append(1) 
for i in X_train_MI:
    y_train_MI.append(1) 


In [264]:
#adding generated data and using twice SMOTE-ENN for sampling
smote_enn = SMOTEENN(random_state = 4)
from collections import Counter
for j in range(95):
    X_train = np.vstack((X_train_MI,X_train_Non_MI,MI_generated_data[:100+10*j]))
    X_test = np.vstack((X_test_MI,X_test_Non_MI))
    X_val = np.vstack((X_val_MI,X_val_Non_MI))


    y_train = np.array(y_train_MI + y_train_Non_MI+ y_generatedMI[:100+10*j])
    y_test = np.array(y_test_MI + y_test_Non_MI)
    y_val = np.array(y_val_MI + y_val_Non_MI)
#first time for selecting sample    
    X_train,y_train = smote_enn.fit_resample(X_train,y_train)
#each time need finetune these two parameters for selection and balance
    if np.sum(y_train==1)>1000 and np.sum(y_train==0)>100:
        print('first time select samples:',sorted(Counter(y_train).items()))
        break
#second time for balance
X_train,y_train = smote_enn.fit_resample(X_train,y_train)   
print('Second time balance samples:',sorted(Counter(y_train).items()))

first time select samples: [(0, 112), (1, 1058)]
Second time balance samples: [(0, 1058), (1, 1057)]


In [250]:
torch_x = torch.from_numpy(X_train)
torch_y = torch.from_numpy(y_train)

test_x = torch.from_numpy(X_test)
test_y = torch.from_numpy(y_test)

test_x_val = torch.from_numpy(X_val)
test_y_val = torch.from_numpy(y_val)

y_train = torch.tensor(torch_y, dtype=torch.long)
y_test = torch.tensor(test_y,dtype = torch.long)
x_test = torch.tensor(test_x , dtype=torch.float32)
x_train = torch.tensor(torch_x, dtype=torch.float32)
x_val = torch.tensor(test_x_val , dtype=torch.float32)
y_val = torch.tensor(test_y_val,  dtype = torch.long)


# In[136]:
torch_x_MI = torch.from_numpy(X_train_MI)
x_train_MI = torch.tensor(torch_x_MI, dtype=torch.float32)
torch_y_MI = torch.from_numpy(np.array(y_train_MI))
y_train_MI = torch.tensor(torch_y_MI, dtype=torch.long)

Non_MI_data = torch.from_numpy(X_test_Non_MI)
MI_data = torch.from_numpy(X_test_MI)
Non_MI_data = torch.tensor(Non_MI_data, dtype=torch.float32)
MI_data = torch.tensor(MI_data, dtype=torch.float32)

Non_MI_label = torch.from_numpy(np.array(y_test_Non_MI))
MI_label = torch.from_numpy(np.array(y_test_MI))
Non_MI_label = torch.tensor(Non_MI_data, dtype=torch.long)
MI_label = torch.tensor(MI_label, dtype=torch.long)


Non_MI_data_val = torch.from_numpy(X_val_Non_MI)
MI_data_val = torch.from_numpy(X_val_MI)
Non_MI_data_val = torch.tensor(Non_MI_data_val, dtype=torch.float32)
MI_data_val = torch.tensor(MI_data_val, dtype=torch.float32)

Non_MI_label_val = torch.from_numpy(np.array(y_val_Non_MI))
MI_label_val = torch.from_numpy(np.array(y_val_MI))
Non_MI_label_val = torch.tensor(Non_MI_data_val, dtype=torch.long)
MI_label_val = torch.tensor(MI_label_val, dtype=torch.long)


d:\softwares\python37\lib\site-packages\ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
d:\softwares\python37\lib\site-packages\ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
d:\softwares\python37\lib\site-packages\ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':
d:\softwares\python37\lib\site-packages\ipykernel_launcher.py:13: UserWarning:

In [251]:
#Hyper parameters
EPOCH = 200
BATCH_SIZE = 64
TIME_STEP = 1
INPUT_SIZE = 382
LR = 0.00005

In [252]:

x_test = x_test.reshape(x_test.shape[0], TIME_STEP, INPUT_SIZE)
x_val = x_val.reshape(x_val.shape[0], TIME_STEP, INPUT_SIZE)
x_train = x_train.reshape(x_train.shape[0], TIME_STEP, INPUT_SIZE)
x_train_MI = x_train_MI.reshape(x_train_MI.shape[0], TIME_STEP, INPUT_SIZE)

Non_MI_data = Non_MI_data.reshape(Non_MI_data.shape[0], TIME_STEP, INPUT_SIZE)
MI_data = MI_data.reshape(MI_data.shape[0], TIME_STEP, INPUT_SIZE)
Non_MI_data_val = Non_MI_data_val.reshape(Non_MI_data_val.shape[0], TIME_STEP, INPUT_SIZE)
MI_data_val = MI_data_val.reshape(MI_data_val.shape[0], TIME_STEP, INPUT_SIZE)

torch_dataset = Data.TensorDataset(x_train,y_train )
train_loader = Data.DataLoader(dataset= torch_dataset, batch_size=BATCH_SIZE, shuffle=True,num_workers=0)


In [253]:
#Model design
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        self.fc  = nn.Sequential(                     
            nn.Linear(382, 383//2),   
            nn.ReLU(True),
            nn.Linear(383//2,383//4),
            nn.ReLU(True),
            nn.Linear(383//4,383//8),
        )
        self.out = nn.Linear(383//8, 2)

    def forward(self, x):
        x = self.fc(x)
        x = x.view(x.size(0), -1)  
        output = self.out(x)
        return output, x  

In [254]:

rnn =  DNN()
optimizer = torch.optim.Adam(rnn.parameters(), lr = LR)
loss_func = torch.nn.BCELoss()


In [255]:
#training model 10 times, and test model with validation dataset

ACC_G_D = float(np.load('ACC_D_G.npy'))
val_list=[]
predict_list = []
for i in range(0,10):
    rnn =  DNN()
    optimizer = torch.optim.Adam(rnn.parameters(), lr = LR)
    loss_func = torch.nn.CrossEntropyLoss()
    
    Loss_CNN = []
    Acc_train = []
    Acc_test = []
    Acc_val = []
    
    Acc_MI = []
    Acc_Non_MI = []
    Times = []
    times = 0
    STEP = []
    
    best_acc = 0
    best_MI = 0
    best_Non_MI = 0

    
    for epoch in range(EPOCH):
        for step, (x, y) in enumerate(train_loader):   
            b_x = Variable(x)
            b_y = Variable(y)   
            output = rnn(b_x)[0]
            output_MI = rnn(x_train_MI)[0]
           
            loss = loss_func(output, b_y)+ACC_G_D*((b_y == 1).sum()/len(y_train))*loss_func(output_MI,y_train_MI)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if step % 50 == 0:
                STEP.append(epoch)
                train_output, last_layer = rnn(x_train)
                pred_y = torch.max(train_output, 1)[1].data.squeeze()
                train_accuracy = (pred_y == y_train).sum().item() / float(y_train.size(0))
 
                MI_data_valout, last_layer = rnn(MI_data_val)
                pred_MI_val = torch.max(MI_data_valout, 1)[1].data.squeeze()
                pred_MI_val = (pred_MI_val.tolist().count(1))/float(MI_data_val.size(0))
                
                Non_MI_data_valout, last_layer = rnn(Non_MI_data_val)
                pred_Non_MI_val = torch.max(Non_MI_data_valout, 1)[1].data.squeeze()
                pred_Non_MI_val = (pred_Non_MI_val.tolist().count(0))/float(Non_MI_data_val.size(0))
                
                
                test_acc =  (pred_MI_val + pred_Non_MI_val)/2

                
                Acc_train.append(train_accuracy)
                Acc_test.append(test_acc)

                
                Acc_MI.append(pred_MI_val)
                Acc_Non_MI.append(pred_Non_MI_val)
                Loss_CNN.append(loss.data)
                times += 1
                Times.append(times)

                if  min(pred_MI_val,pred_Non_MI_val)>= 0.6 and test_acc>best_acc:
                    
                    best_acc  = test_acc
                    best_MI = pred_MI_val
                    best_Non_MI = pred_Non_MI_val
                    
                    torch.save(rnn,'dnnL_best_{}'.format(i)+'.pkl')
                    
    if best_acc == 0:
        best_acc  = test_acc
        best_MI = pred_MI_val
        best_Non_MI = pred_Non_MI_val

        torch.save(rnn,'dnnL_best_{}'.format(i)+'.pkl')
        
    np.save('Acc_train_best_{}'.format(i)+'.npy',Acc_train)
    np.save('Acc_val_best_{}'.format(i)+'.npy',Acc_val)
    
    np.save('Acc_testave_final_{}'.format(i)+'.npy',test_acc)
    np.save('Acc_val_hf_{}'.format(i)+'.npy',pred_Non_MI_val)
    np.save('Acc_val_sf_{}'.format(i)+'.npy',pred_MI_val)
    
    np.save('pred_Non_MI_dnnL_end_{}'.format(i)+'.npy',Acc_Non_MI)
    np.save('Acc_train_dnnL_end_{}'.format(i)+'.npy',Acc_train)
    np.save('Loss_dnnL_end_{}'.format(i)+'.npy',Loss_CNN)
    np.save('STEP_dnnL_end_{}'.format(i)+'.npy',STEP)
    np.save('Times_dnnL_end_{}'.format(i)+'.npy',Times)
    val_list.append(best_acc)
    print('round:',i,',val_acc:',best_acc,',val_MI:',best_MI,',val_Non_MI:',best_Non_MI)


round: 0 ,val_acc: 0.7498383968972204 ,val_MI: 0.8571428571428571 ,val_Non_MI: 0.6425339366515838
round: 1 ,val_acc: 0.7521008403361344 ,val_MI: 0.8571428571428571 ,val_Non_MI: 0.6470588235294118
round: 2 ,val_acc: 0.7521008403361344 ,val_MI: 0.8571428571428571 ,val_Non_MI: 0.6470588235294118
round: 3 ,val_acc: 0.77650290885585 ,val_MI: 0.9285714285714286 ,val_Non_MI: 0.6244343891402715
round: 4 ,val_acc: 0.7340012928248223 ,val_MI: 0.8571428571428571 ,val_Non_MI: 0.6108597285067874
round: 5 ,val_acc: 0.7923400129282483 ,val_MI: 0.9285714285714286 ,val_Non_MI: 0.6561085972850679
round: 6 ,val_acc: 0.7475759534583064 ,val_MI: 0.8571428571428571 ,val_Non_MI: 0.6380090497737556
round: 7 ,val_acc: 0.7430510665804784 ,val_MI: 0.8571428571428571 ,val_Non_MI: 0.6289592760180995
round: 8 ,val_acc: 0.7430510665804784 ,val_MI: 0.8571428571428571 ,val_Non_MI: 0.6289592760180995
round: 9 ,val_acc: 0.7412734324499031 ,val_MI: 0.7857142857142857 ,val_Non_MI: 0.6968325791855203


In [256]:
#testing model with test dataset

dnn_acc = []
dnn_MI = []
dnn_Non_MI = []

for i in range(10):
    rnn = torch.load('dnnL_best_{}'.format(i)+'.pkl')
    
    Non_MI_data_out, last_layer = rnn(Non_MI_data)
    pred_Non_MI = torch.max(Non_MI_data_out, 1)[1].data.squeeze()
    pred_Non_MI = (pred_Non_MI.tolist().count(0))/float(Non_MI_data.size(0))
    MI_data_out, last_layer = rnn(MI_data)
    pred_MI = torch.max(MI_data_out, 1)[1].data.squeeze()
    pred_MI = (pred_MI.tolist().count(1))/float(MI_data.size(0))
    predict_acc = (pred_Non_MI+pred_MI)/2 
    
    dnn_acc.append(predict_acc)
    dnn_MI.append(pred_MI)
    dnn_Non_MI.append(pred_Non_MI)
    

In [261]:
#calculate the average
dnn_acc_mean = round(sum(dnn_acc)/10,2)
dnn_MI_mean = round(sum(dnn_MI)/10,2)
dnn_Non_MI_mean = round(sum(dnn_Non_MI)/10,2)

In [262]:

print('final accuracy:',dnn_acc_mean)
print('final MI accuracy:',dnn_MI_mean)
print('final Non-MI accuracy:',dnn_Non_MI_mean)

final accuracy: 0.72
final MI accuracy: 0.84
final Non-MI accuracy: 0.6


In [ ]:
#this is only one results of RS=32 from K-means, each time needs to finetune the SMOTE-ENN step, and after calculating, the average is 70%